In [ ]:
!pip install -U accelerate -q
!pip install -U transformers -q
!pip install transformers accelerate bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"
prompt = "Tell me about Donald Trump"

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tell me about Donald Trump’s tax plan. Einzeln
Donald Trump has proposed a comprehensive tax reform plan that he claims will help America’s middle class and small businesses. Here are some key elements of his plan:
1. Lowering the business tax rate: Trump wants to reduce the business tax rate from 35% to 15%. This would make the US more competitive with other countries and encourage businesses to invest and hire more workers.
2. Doubling the standard deduction: Trump wants to increase the standard deduction from $12,000 to $24,000 for individuals and from $24,000 to $48,000 for married couples. This would help reduce the amount of taxes that people pay on their income.
3. Eliminating the estate tax: Trump wants to repeal the estate tax, which is also known as the “death tax.” This tax is levied on the estates of people who have died, and Trump claims it unfairly targets small businesses and farms.
4. Capping the charitable donation deduction: Trump wants to limit the amount that people

In [ ]:
output = model.generate(**model_inputs)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Tell me about Donald Trump's "Art of the Deal"
 Unterscheidung zwischen "Art of the Deal" und einem anderen Buch über den "Deal"

Donald Trump's "Art of the Deal" is a book written by the 45th President of the United States, Donald Trump, and published in 1987. The book is a self-help and business advice book that offers Trump's insights and strategies for success in business and life.
The book is divided into 12 chapters, each focusing on a different aspect of deal-making, such as negotiating, marketing, and closing deals. Trump shares his own experiences and anecdotes from his business career, offering practical advice and tips for readers looking to improve their negotiation and deal-making skills.
One of the key themes of the book is the importance of being a strong negotiator. Trump emphasizes the need to be confident, assertive, and willing to walk away from a bad deal. He also stresses the importance of having a clear vision and a well-defined strategy for achieving one's goals.

In [ ]:
prompt = "France has a bread law, Le Décret Pain, with strict rules on what is allowed in a traditional baguette."
candidate1 = "The law does not apply to croissants and brioche."
candidate2 = "The law applies to baguettes."

In [ ]:
from transformers import AutoModelForMultipleChoice
model = AutoModelForMultipleChoice.from_pretrained(model_name, device_map="auto", load_in_4bit=True)

ValueError: ignored

In [ ]:
import torch
model_inputs = tokenizer([[prompt, candidate1], [prompt, candidate2]], return_tensors="pt").to("cuda:0")

labels = torch.tensor(0).unsqueeze(0)

ValueError: ignored

In [ ]:
result.logits

tensor([[[ -0.0486,  -1.7080,   0.0869,  ...,   1.3975,   1.5039,   0.4370],
         [ -8.7109,  -8.6172,  -0.1006,  ...,  -4.1211,  -5.8516,  -5.6562],
         [ -0.7783,   1.2002,   5.8320,  ...,   0.7549,  -1.4062,  -1.4502],
         [-10.8672,  -8.8438,  -3.9238,  ...,  -5.2109,  -7.3867,  -8.2344],
         [ -4.6289,  -6.1445,   4.0156,  ...,  -2.9121,  -3.1699,  -5.3320],
         [ -1.1934,  -1.6387,  11.0547,  ...,  -0.0321,  -0.2375,   1.5273]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)

In [ ]:
model_inputs

{'input_ids': tensor([[    1, 24948,   592,  1048, 20953]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}